<a href="https://colab.research.google.com/github/Hamidreza-Atri/Thyroid_Image_Generation/blob/main/Thyroid_Image_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirements

In [ ]:
import os
import pandas as pd
from PIL import Image
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
from matplotlib.pyplot import imshow
from matplotlib import pyplot
from os import makedirs
from keras import layers
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import choice
from numpy.random import randn
from numpy.random import random
from numpy.random import randint
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Input
from keras.models import Model
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.initializers import RandomNormal
from keras.utils.vis_utils import plot_model
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# Pre processing

In [ ]:
# List image names and their address
path = "/content/drive/MyDrive/Colab_Notebooks/Thyroid/img"
img_names=[]
for folder,subfolders,filenames in os.walk(path):
  for img in filenames:
    img_names.append(folder+"/"+img)
# Resizing images
temp_list=[]
first_process=[]
input_images=[]
tool=len(img_names)

IMG_SIZE=28
for i in range(tool):

  first_process = tf.io.read_file(img_names[i])
  first_process = tf.image.decode_png(first_process, channels=1, dtype=tf.dtypes.uint8)
  # first_process = tf.image.convert_image_dtype(first_process, tf.float32)
  first_process = tf.image.resize(first_process, size=[IMG_SIZE, IMG_SIZE])
  # first_process = (first_process - 127.5) / 127.5
  temp_list.append(first_process)

input_images=np.array(temp_list)
print(input_images.shape)

# imshow(input_images[25,:,:,0], cmap='gray_r')

Importing data

In [ ]:
# Importing data labels
data=pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/Thyroid/data/thyroid_new_1.csv")
SQdata=np.squeeze(data,axis=1)
image_labels=np.asarray(SQdata, dtype=np.uint8, order=None)

# Network implementation

In [ ]:
# defining discriminator model
def define_discriminator(in_shape=(28,28,1), n_classes=3):
	# weight initialization
  init = RandomNormal(mean=0.0, stddev=0.02)
	# image input
  in_image = Input(shape=in_shape)
	# downsample to 14x14
  fe = Conv2D(32, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
  fe = LeakyReLU(alpha=0.2)(fe)
  fe = Dropout(0.5)(fe)
	# normal
  fe = Conv2D(64, (3,3), padding='same', kernel_initializer=init)(fe)
  fe = BatchNormalization()(fe)
  fe = LeakyReLU(alpha=0.2)(fe)
  fe = Dropout(0.5)(fe)
	# downsample to 7x7
  fe = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(fe)
  fe = BatchNormalization()(fe)
  fe = LeakyReLU(alpha=0.2)(fe)
  fe = Dropout(0.5)(fe)
	# normal
  fe = Conv2D(256, (3,3), padding='same', kernel_initializer=init)(fe)
  fe = BatchNormalization()(fe)
  fe = LeakyReLU(alpha=0.2)(fe)
  fe = Dropout(0.5)(fe)
 	# flatten feature maps
  fe = Flatten()(fe)
	# real/fake output
  out1 = Dense(1, activation='sigmoid')(fe)
	# class label output
  out2 = Dense(n_classes, activation='softmax')(fe)
	# define model
  model = Model(in_image, [out1, out2])
	# compile model
  opt = Adam(lr=0.0002, beta_1=0.5)
  model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opt, metrics=['accuracy'])
  return model

# defining generator model
def define_generator(latent_dim, n_classes=3):
	# weight initialization
  init = RandomNormal(mean=0.0, stddev=0.02)
	# label input
  in_label = Input(shape=(1,))
	# embedding for categorical input
  li = Embedding(n_classes, 50)(in_label)
	# linear multiplication
  n_nodes = 7 * 7
  li = Dense(n_nodes, kernel_initializer=init)(li)
	# reshape to additional channel
  li = Reshape((7, 7, 1))(li)
	# image generator input
  in_lat = Input(shape=(latent_dim,))
	# foundation for 7x7 image
  n_nodes = 384 * 7 * 7
  gen = Dense(n_nodes, kernel_initializer=init)(in_lat)
  gen = Activation('relu')(gen)
  # gen = Activation('relu')(gen)
  gen = Reshape((7, 7, 384))(gen)
	# merge image gen and label input
  merge = Concatenate()([gen, li])
	# upsample to 14x14
  gen = Conv2DTranspose(192, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(merge)
  gen = BatchNormalization()(gen)
  gen = Activation('relu')(gen)
  # upsample to 28x28
  gen = Conv2DTranspose(1, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(gen)
  out_layer = Activation('tanh')(gen)
	# define model
  model = Model([in_lat, in_label], out_layer)
  return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	for layer in d_model.layers:
		if not isinstance(layer, BatchNormalization):
			layer.trainable = False
	# connect the outputs of the generator to the inputs of the discriminator
	gan_output = d_model(g_model.output)
	# define gan model as taking noise and label and outputting real/fake and label outputs
	model = Model(g_model.input, gan_output)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opt)
	return model

# load images
def load_real_samples():
	# load images and labels
  (X, trainy) = input_images,image_labels
	# convert from ints to floats
  X = X.astype('float32')
	# scale from [0,255] to [-1,1]
  X = (X - 127.5) / 127.5
  print(X.shape, trainy.shape)
  return [X, trainy]

# select real samples
def generate_real_samples(dataset, n_samples):
	# split into images and labels
	images, labels = dataset
	# choose random instances
	ix = randint(0, images.shape[0], n_samples)
	# select images and labels
	X, labels = images[ix], labels[ix]
	# generate class labels
	y = ones((n_samples, 1))
	return [X, labels], y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=3):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = x_input.reshape(n_samples, latent_dim)
	# generate labels
	labels = randint(0, n_classes, n_samples)
	return [z_input, labels]

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	z_input, labels_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	images = generator.predict([z_input, labels_input])
	# create class labels
	y = zeros((n_samples, 1))
	return [images, labels_input], y

# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, latent_dim, n_samples=100):
	# prepare fake examples
	[X, _], _ = generate_fake_samples(g_model, latent_dim, n_samples)
	# scale from [-1,1] to [0,1]
	X = (X + 1) / 2.0
	# plot images
	for i in range(100):
		# define subplot
		pyplot.subplot(10, 10, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
	# save plot to file
	filename1 = '/content/drive/MyDrive/Colab_Notebooks/Thyroid/Output/generated_plot_%04d.png' % (step+1)
	pyplot.savefig(filename1)
	pyplot.close()
	# save the generator model
	filename2 = '/content/drive/MyDrive/Colab_Notebooks/Thyroid/Output/model_%04d.h5' % (step+1)
	g_model.save(filename2)
	print('>Saved: %s and %s' % (filename1, filename2))
 
def plot_history(d1_hist, d4_hist, g1_hist, a4_hist, a3_hist, step):
	# plot loss
  pyplot.subplot(2, 1, 1)
  pyplot.plot(d1_hist, label='d-real')
  pyplot.plot(d4_hist, label='d-fake')
  pyplot.plot(g1_hist, label='gen')
  pyplot.legend()
	# plot discriminator accuracy
  pyplot.subplot(2, 1, 2)
  pyplot.plot(a4_hist, label='acc-real')
  pyplot.plot(a3_hist, label='acc-fake')
  pyplot.legend()
	# save plot to file
	# pyplot.savefig('results_opt/plot_line_plot_loss.png')
  filename3 = '/content/drive/MyDrive/Colab_Notebooks/Thyroid/Output/generated_plot_loss_%04d.png' % (step+1)
  pyplot.savefig(filename3)
  pyplot.close()

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=10000, n_batch=100):#500_100 #3000->15000 / 10000->50000
	# calculate the number of batches per training epoch
  bat_per_epo = int(dataset[0].shape[0] / n_batch)
	# calculate the number of training iterations
  n_steps = bat_per_epo * n_epochs
	# calculate the size of half a batch of samples
  half_batch = int(n_batch / 2)
  s=0
  d1_hist, d2_hist, d3_hist, d4_hist, g1_hist,g2_hist, a1_hist, a2_hist, a3_hist, a4_hist = list(), list(), list(), list(), list(), list(), list(), list(), list(), list()
	# manually enumerate epochs
  for i in range(n_steps):
		# get randomly selected 'real' samples
    [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
		# update discriminator model weights
    _,dr_i,dr_l,accdr_i,accdr_l = d_model.train_on_batch(X_real, [y_real, labels_real])
		# generate 'fake' examples
    [X_fake, labels_fake], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
		# update discriminator model weights
    _,df_i,df_l,accdf_i,accdf_l = d_model.train_on_batch(X_fake, [y_fake, labels_fake])
		# prepare points in latent space as input for the generator
    [z_input, z_labels] = generate_latent_points(latent_dim, n_batch)
		# create inverted labels for the fake samples
    y_gan = ones((n_batch, 1))
		# update the generator via the discriminator's error
    _,g_1,g_2 = gan_model.train_on_batch([z_input, z_labels], [y_gan, z_labels])
		# summarize loss on this batch
    print('>%d, dr=[%.3f,%.3f], accdr[%.3f,%.3f], df[%.3f,%.3f], accdf[%.3f,%.3f], g[%.3f,%.3f]' % (i+1,dr_i,dr_l,accdr_i,accdr_l,df_i,df_l,accdf_i,accdf_l, g_1,g_2))
    # record history
    d1_hist.append(dr_i)
    d2_hist.append(dr_l)
    d3_hist.append(df_l)
    d4_hist.append(df_i)
    g1_hist.append(g_1)
    g2_hist.append(g_2)
    a1_hist.append(accdr_l)
    a2_hist.append(accdf_l)
    a3_hist.append(accdf_i)
    a4_hist.append(accdr_i)
    s=i
		# evaluate the model performance every 'epoch'
    # if (i+1) % (bat_per_epo * 10) == 0:
  summarize_performance(i, g_model, latent_dim)

  plot_history(d1_hist, d4_hist, g1_hist, a4_hist, a3_hist, s)
  print('\n')
  print("**Discrenimator_Loss**: ")
  print("Real_Data_Image_Loss: ",int(100*(np.mean(d1_hist))),"%")
  print("Fake_Data_Image_Loss: ",int(100*(np.mean(d4_hist))),"%")
  print("Real_Data_Label_Loss: ",int(100*(np.mean(d2_hist))),"%")
  print("Fake_Data_Label_Loss: ",int(100*(np.mean(d3_hist))),"%",'\n')
  print("**Generator_Loss**: ")
  print("Generated_Image_Loss: ",int(100*(np.mean(g1_hist))),"%")
  print("Generated_label_Loss: ",int(100*(np.mean(g2_hist))),"%",'\n')
  print("**Accuarcy**: ")
  print("Sum_Accuarcy_Discrenimator_Real_Data: ",int(100*(np.mean(a1_hist+a4_hist))),"%")
  print("Sum_Accuarcy_Discrenimator_Fake_Data: ",int(100*(np.mean(a2_hist+a3_hist))),"%")
  print("Accuarcy_Discrenimator_Real_Label: ",int(100*(np.mean(a1_hist))),"%")
  print("Accuarcy_Discrenimator_Fake_Label: ",int(100*(np.mean(a2_hist))),"%")
  print("Accuarcy_Discrenimator_Real_Image: ",int(100*(np.mean(a4_hist))),"%")
  print("Accuarcy_Discrenimator_Fake_Image: ",int(100*(np.mean(a3_hist))),"%")

# size of the latent space
latent_dim = 110
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# load image data
dataset = load_real_samples()
# train model
train(generator, discriminator, gan_model, dataset, latent_dim)

# Model Evaluation

In [ ]:
from math import sqrt
from numpy import asarray
from tensorflow import keras
from numpy.random import randn
from keras.models import load_model
from matplotlib import pyplot
from datetime import datetime
import tensorflow as tf

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_class):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = x_input.reshape(n_samples, latent_dim)
	# generate labels
	labels = asarray([n_class for _ in range(n_samples)])
	return [z_input, labels]
# p2=np.array([])
# create and save a plot of generated images
def save_plot(examples, n_examples):
	# plot images
  for i in range(n_examples):
		# define subplot
    pyplot.subplot(sqrt(n_examples), sqrt(n_examples), 1 + i)
		# turn off axis
    pyplot.axis('off')
		# plot raw pixel data
    pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
    # p2=examples[i, :, :, 0]
   
  filename1 = 'generated_pic_100000_1.png' #% (step+1)
  pyplot.savefig(filename1,dpi=None,facecolor=None,edgecolor=None,transparent=True,bbox_inches=None,
         metadata=None)
  pyplot.show()

# load model
model = load_model('model_50000.h5')
latent_dim = 110
n_examples =1  # must be a square
n_class = 1 # sneaker
# generate images
latent_points, labels = generate_latent_points(latent_dim, n_examples, n_class)
# generate images
X  = model.predict([latent_points, labels])#,callbacks=[tensorboard_callback])
# scale from [-1,1] to [0,1]
X = (X + 1) / 2.0
# plot the result
save_plot(X, n_examples)

# Image blending

In [ ]:
# Import requierments
import skimage
import numpy as np
from PIL import Image
from skimage import io
from skimage import data
import matplotlib.pyplot as plt

# Scikit-image code for creating round faded circle around images
input_image = skimage.io.imread("001.png")
print(input_image,'Thyroid')
l_row, l_col, nb_channel = input_image.shape
print(input_image.shape,'input_image.shape)')
rows, cols = np.mgrid[:l_row, :l_col]
radius = np.sqrt((rows - l_row/2)**2 + (cols - l_col/2)**2)
print(radius,'radius')
alpha_channel = np.zeros((l_row, l_col))
print(alpha_channel,'alpha_channel')
r_min, r_max = 1./3 * radius.max(), 0.8 * radius.max()
print(r_min,'r_min')
print(r_max,'r_max')
alpha_channel[radius < r_min] = 1
alpha_channel[radius > r_max] = 0
gradient_zone = np.logical_and(radius >= r_min, radius <= r_max)
alpha_channel[gradient_zone] = (r_max - radius[gradient_zone])/(r_max - r_min)
alpha_channel *= 255
feathered = np.empty((l_row, l_col, nb_channel + 1), dtype=np.uint8)
feathered[..., :3] = input_image[:]
feathered[..., -1] = alpha_channel[:]
# Output image
data = np.random.rand(256,256)
plt.imshow(feathered)
plt.imsave('Feathered_image.png', feathered, cmap = plt.cm.gray)

# Data Evaluation

In [ ]:
# calculating the frechet inception distance
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import randint
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data
from skimage.transform import resize
from PIL import Image

# scale an array of images to a new size
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return asarray(images_list)

# calculate frechet inception distance
def calculate_fid(model, images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

# prepare the inception v3 model
model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))
# define two collections of numpy images to compare with eathother
images1 = numpy.array(Image.open("/content/New1.png"))
images2 = numpy.array(Image.open("/content/New2.png"))
print('Prepared', images1.shape, images2.shape)
# convert integer to floating point values
images1 = images1.astype('float32')
images2 = images2.astype('float32')
# resize images
images1 = scale_images(images1, (299,299,3))
images2 = scale_images(images2, (299,299,3))
print('Scaled', images1.shape, images2.shape)
# pre-process images
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)
# fid between images1 and images1
fid = calculate_fid(model, images1, images1)
print('FID (same): %.3f' % fid)
# fid between images1 and images2
fid = calculate_fid(model, images1, images2)
print('FID (different): %.3f' % fid)